# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from fastai.vision.all import *
path = Path('/home/gegg00/Desktop/aiornot/')
df = pd.read_csv(path/'train.csv')

In [4]:
#| export
def get_images(path):
    return [path/'data/train'/i[0] for i in df.values]

def get_label(o):
    return df.iloc[Int(os.path.basename(o).split('.')[0])]['label'] 

In [5]:
get_images(path)

[Path('/home/gegg00/Desktop/aiornot/data/train/0.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/1.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/2.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/3.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/4.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/5.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/6.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/7.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/8.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/9.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/10.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/11.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/12.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/13.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/14.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/15.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/train/16.jpg'),
 Path('/home/gegg00/Desktop/aiornot/data/

In [6]:
#| export
tst_files = get_image_files(path/'data/test').sorted()

In [7]:
def get_dls():
    dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock), 
        get_items=get_images, 
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=get_label,
        item_tfms=[Resize(size, method='squish')],
        batch_tfms=aug_transforms(size=size, min_scale=0.75)
    ).dataloaders(path, bs=64//accum)

In [8]:
#| export
def train(arch, size, accum=1, finetune=True, epochs=12):
    dls = DataBlock(
        blocks=(ImageBlock, CategoryBlock), 
        get_items=get_images, 
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=get_label,
        item_tfms=[Resize(size, method='squish')],
        batch_tfms=aug_transforms(size=size, min_scale=0.75)
    ).dataloaders(path, bs=64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)

In [9]:
#| export
from fastcore.script import *
@call_parse

def main(arch:str,   #Model to use from timm 
         size:int,  #size of the images
         accum:int, #size of the batch
         epochs:int, #number of epochs
         finetune:bool):  #Fine tune model
    print(train(arch, size, accum, finetune, epochs=epochs))
    #tst_files = get_image_files(path/'data/test').sorted()
    #tst_dl = dls.test_dl(tst_files)

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()